In [14]:
import polars as pl

CSV_OPTS = dict(
    null_values=["NA"],
    ignore_errors=False,
    infer_schema_length=10000
)

airlines = pl.read_csv(
    'https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airlines.csv',
    **CSV_OPTS
)
airports = pl.read_csv(
    'https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airports.csv',
    **CSV_OPTS
)
flights  = pl.read_csv(
    'https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_flights.csv',
    **CSV_OPTS
)
planes   = pl.read_csv(
    'https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_planes.csv',
    **CSV_OPTS
)
weather  = pl.read_csv(
    'https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_weather.csv',
    **CSV_OPTS
)

flights = flights.with_columns(pl.col("time_hour").str.strptime(pl.Datetime, strict=False))
weather = weather.with_columns(pl.col("time_hour").str.strptime(pl.Datetime, strict=False))

ctx = pl.SQLContext(
    airlines=airlines,
    airports=airports,
    flights=flights,
    planes=planes,
    weather=weather,
    eager_execution=True,
)

print("Setup complete! Tables available:")
print(ctx.execute("SHOW TABLES"))

Setup complete! Tables available:
shape: (5, 1)
┌──────────┐
│ name     │
│ ---      │
│ str      │
╞══════════╡
│ airlines │
│ airports │
│ flights  │
│ planes   │
│ weather  │
└──────────┘


/tmp/ipython-input-3825821825.py:33: DeprecationWarning: The argument `eager_execution` for `SQLContext.__init__` is deprecated. It has been renamed to `eager`.
  ctx = pl.SQLContext(


In [15]:
result = ctx.execute("""
SELECT DISTINCT carrier, name
FROM airlines
ORDER BY carrier
""")
print(result)

shape: (16, 2)
┌─────────┬────────────────────────┐
│ carrier ┆ name                   │
│ ---     ┆ ---                    │
│ str     ┆ str                    │
╞═════════╪════════════════════════╡
│ 9E      ┆ Endeavor Air Inc.      │
│ AA      ┆ American Airlines Inc. │
│ AS      ┆ Alaska Airlines Inc.   │
│ B6      ┆ JetBlue Airways        │
│ DL      ┆ Delta Air Lines Inc.   │
│ …       ┆ …                      │
│ UA      ┆ United Air Lines Inc.  │
│ US      ┆ US Airways Inc.        │
│ VX      ┆ Virgin America         │
│ WN      ┆ Southwest Airlines Co. │
│ YV      ┆ Mesa Airlines Inc.     │
└─────────┴────────────────────────┘


In [16]:
result = ctx.execute("""
SELECT
  dest,
  COUNT(*) AS num_flights
FROM flights
GROUP BY dest
ORDER BY num_flights DESC
LIMIT 10
""")
print(result)

shape: (10, 2)
┌──────┬─────────────┐
│ dest ┆ num_flights │
│ ---  ┆ ---         │
│ str  ┆ u32         │
╞══════╪═════════════╡
│ ORD  ┆ 17283       │
│ ATL  ┆ 17215       │
│ LAX  ┆ 16174       │
│ BOS  ┆ 15508       │
│ MCO  ┆ 14082       │
│ CLT  ┆ 14064       │
│ SFO  ┆ 13331       │
│ FLL  ┆ 12055       │
│ MIA  ┆ 11728       │
│ DCA  ┆ 9705        │
└──────┴─────────────┘


In [17]:
result = ctx.execute("""
SELECT
  year, month, day, dep_time, sched_dep_time,
  dep_delay, carrier, flight, origin, dest, tailnum
FROM flights
WHERE dep_delay > 120
ORDER BY dep_delay DESC
""")
print(result)

shape: (9_723, 11)
┌──────┬───────┬─────┬──────────┬───┬────────┬────────┬──────┬─────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ flight ┆ origin ┆ dest ┆ tailnum │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---    ┆ ---    ┆ ---  ┆ ---     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64    ┆ str    ┆ str  ┆ str     │
╞══════╪═══════╪═════╪══════════╪═══╪════════╪════════╪══════╪═════════╡
│ 2013 ┆ 1     ┆ 9   ┆ 641      ┆ … ┆ 51     ┆ JFK    ┆ HNL  ┆ N384HA  │
│ 2013 ┆ 6     ┆ 15  ┆ 1432     ┆ … ┆ 3535   ┆ JFK    ┆ CMH  ┆ N504MQ  │
│ 2013 ┆ 1     ┆ 10  ┆ 1121     ┆ … ┆ 3695   ┆ EWR    ┆ ORD  ┆ N517MQ  │
│ 2013 ┆ 9     ┆ 20  ┆ 1139     ┆ … ┆ 177    ┆ JFK    ┆ SFO  ┆ N338AA  │
│ 2013 ┆ 7     ┆ 22  ┆ 845      ┆ … ┆ 3075   ┆ JFK    ┆ CVG  ┆ N665MQ  │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …      ┆ …      ┆ …    ┆ …       │
│ 2013 ┆ 9     ┆ 2   ┆ 2302     ┆ … ┆ 1103   ┆ JFK    ┆ SJU  ┆ N618JB  │
│ 2013 ┆ 9     ┆ 11  ┆ 1850     ┆ … ┆ 5940   ┆ EWR    ┆ SAV  ┆ N13968  │
│ 2013 ┆ 9     ┆ 12  ┆ 1700     

In [18]:
result = ctx.execute("""
SELECT
  origin,
  AVG(dep_delay) AS avg_dep_delay
FROM flights
WHERE dep_delay IS NOT NULL
GROUP BY origin
ORDER BY avg_dep_delay DESC
""")
print(result)

shape: (3, 2)
┌────────┬───────────────┐
│ origin ┆ avg_dep_delay │
│ ---    ┆ ---           │
│ str    ┆ f64           │
╞════════╪═══════════════╡
│ EWR    ┆ 15.107954     │
│ JFK    ┆ 12.112159     │
│ LGA    ┆ 10.346876     │
└────────┴───────────────┘


In [19]:
result = ctx.execute("""
SELECT
  month,
  COUNT(*) AS num_flights
FROM flights
GROUP BY month
ORDER BY num_flights DESC
LIMIT 1
""")
print(result)

shape: (1, 2)
┌───────┬─────────────┐
│ month ┆ num_flights │
│ ---   ┆ ---         │
│ i64   ┆ u32         │
╞═══════╪═════════════╡
│ 7     ┆ 29425       │
└───────┴─────────────┘


In [22]:
result = ctx.execute("""
WITH base AS (
  SELECT
    f.carrier,
    a.name AS airline_name,
    COUNT(*) AS total_flights,
    SUM(CASE WHEN f.dep_delay <= 15 THEN 1 ELSE 0 END) AS on_time_flights
  FROM flights AS f
  LEFT JOIN airlines AS a
    ON a.carrier = f.carrier
  WHERE f.dep_delay IS NOT NULL
  GROUP BY f.carrier, a.name
)
SELECT
  carrier,
  airline_name,
  total_flights,
  on_time_flights,
  100.0 * on_time_flights / NULLIF(total_flights, 0) AS on_time_rate_pct
FROM base
ORDER BY on_time_rate_pct DESC
""")
print(result)

shape: (16, 5)
┌─────────┬─────────────────────────────┬───────────────┬─────────────────┬──────────────────┐
│ carrier ┆ airline_name                ┆ total_flights ┆ on_time_flights ┆ on_time_rate_pct │
│ ---     ┆ ---                         ┆ ---           ┆ ---             ┆ ---              │
│ str     ┆ str                         ┆ u32           ┆ i32             ┆ f64              │
╞═════════╪═════════════════════════════╪═══════════════╪═════════════════╪══════════════════╡
│ HA      ┆ Hawaiian Airlines Inc.      ┆ 342           ┆ 318             ┆ 92.982456        │
│ US      ┆ US Airways Inc.             ┆ 19873         ┆ 17453           ┆ 87.822674        │
│ AS      ┆ Alaska Airlines Inc.        ┆ 712           ┆ 618             ┆ 86.797753        │
│ AA      ┆ American Airlines Inc.      ┆ 32093         ┆ 26981           ┆ 84.071293        │
│ DL      ┆ Delta Air Lines Inc.        ┆ 47761         ┆ 39967           ┆ 83.681246        │
│ …       ┆ …                      

In [23]:
result = ctx.execute("""
SELECT
  f.year, f.month, f.day,
  f.dep_time, f.arr_time,
  f.carrier, a.name AS airline_name,
  f.flight, f.origin, f.dest, f.tailnum
FROM flights AS f
LEFT JOIN airlines AS a
  ON a.carrier = f.carrier
ORDER BY f.year, f.month, f.day, f.dep_time NULLS LAST
LIMIT 100
""")
print(result)

shape: (100, 11)
┌──────┬───────┬─────┬──────────┬───┬────────┬────────┬──────┬─────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ flight ┆ origin ┆ dest ┆ tailnum │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---    ┆ ---    ┆ ---  ┆ ---     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64    ┆ str    ┆ str  ┆ str     │
╞══════╪═══════╪═════╪══════════╪═══╪════════╪════════╪══════╪═════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 517      ┆ … ┆ 1545   ┆ EWR    ┆ IAH  ┆ N14228  │
│ 2013 ┆ 1     ┆ 1   ┆ 533      ┆ … ┆ 1714   ┆ LGA    ┆ IAH  ┆ N24211  │
│ 2013 ┆ 1     ┆ 1   ┆ 542      ┆ … ┆ 1141   ┆ JFK    ┆ MIA  ┆ N619AA  │
│ 2013 ┆ 1     ┆ 1   ┆ 544      ┆ … ┆ 725    ┆ JFK    ┆ BQN  ┆ N804JB  │
│ 2013 ┆ 1     ┆ 1   ┆ 554      ┆ … ┆ 461    ┆ LGA    ┆ ATL  ┆ N668DN  │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …      ┆ …      ┆ …    ┆ …       │
│ 2013 ┆ 1     ┆ 1   ┆ 746      ┆ … ┆ 1668   ┆ EWR    ┆ SFO  ┆ N24224  │
│ 2013 ┆ 1     ┆ 1   ┆ 749      ┆ … ┆ 3737   ┆ EWR    ┆ ORD  ┆ N508MQ  │
│ 2013 ┆ 1     ┆ 1   ┆ 752      ┆ 

In [26]:
result = ctx.execute("""
SELECT
  f.carrier,
  al.name AS airline_name,
  AVG(f.year - p.year) AS avg_plane_age_years,
  COUNT(*) AS flights_count
FROM flights AS f
LEFT JOIN planes AS p
  ON p.tailnum = f.tailnum
LEFT JOIN airlines AS al
  ON al.carrier = f.carrier
WHERE p.year IS NOT NULL    -- only when plane build year is known
GROUP BY f.carrier, al.name
HAVING COUNT(*) > 0
ORDER BY avg_plane_age_years DESC
""")
print(result)

shape: (16, 4)
┌─────────┬─────────────────────────────┬─────────────────────┬───────────────┐
│ carrier ┆ airline_name                ┆ avg_plane_age_years ┆ flights_count │
│ ---     ┆ ---                         ┆ ---                 ┆ ---           │
│ str     ┆ str                         ┆ f64                 ┆ u32           │
╞═════════╪═════════════════════════════╪═════════════════════╪═══════════════╡
│ MQ      ┆ Envoy Air                   ┆ 35.319              ┆ 1000          │
│ AA      ┆ American Airlines Inc.      ┆ 25.869426           ┆ 9979          │
│ DL      ┆ Delta Air Lines Inc.        ┆ 16.372169           ┆ 47473         │
│ UA      ┆ United Air Lines Inc.       ┆ 13.207691           ┆ 56117         │
│ FL      ┆ AirTran Airways Corporation ┆ 11.385829           ┆ 2978          │
│ …       ┆ …                           ┆ …                   ┆ …             │
│ B6      ┆ JetBlue Airways             ┆ 6.686702            ┆ 52956         │
│ F9      ┆ Frontier Airl

In [27]:
result = ctx.execute("""
SELECT
  f.year, f.month, f.day, f.hour,
  f.origin, f.dest, f.dep_delay,
  w.wind_speed, w.precip
FROM flights AS f
JOIN weather AS w
  ON w.origin = f.origin
 AND w.time_hour = f.time_hour
WHERE f.dep_delay > 30
  AND (w.wind_speed > 20 OR w.precip > 0.1)
ORDER BY f.dep_delay DESC
LIMIT 100
""")
print(result)

shape: (100, 9)
┌──────┬───────┬─────┬──────┬───┬──────┬───────────┬────────────┬────────┐
│ year ┆ month ┆ day ┆ hour ┆ … ┆ dest ┆ dep_delay ┆ wind_speed ┆ precip │
│ ---  ┆ ---   ┆ --- ┆ ---  ┆   ┆ ---  ┆ ---       ┆ ---        ┆ ---    │
│ i64  ┆ i64   ┆ i64 ┆ i64  ┆   ┆ str  ┆ i64       ┆ f64        ┆ f64    │
╞══════╪═══════╪═════╪══════╪═══╪══════╪═══════════╪════════════╪════════╡
│ 2013 ┆ 4     ┆ 10  ┆ 19   ┆ … ┆ TPA  ┆ 960       ┆ 31.07106   ┆ 0.11   │
│ 2013 ┆ 12    ┆ 14  ┆ 18   ┆ … ┆ TPA  ┆ 825       ┆ 20.71404   ┆ 0.01   │
│ 2013 ┆ 4     ┆ 19  ┆ 17   ┆ … ┆ LAS  ┆ 797       ┆ 25.31716   ┆ 0.0    │
│ 2013 ┆ 4     ┆ 19  ┆ 17   ┆ … ┆ IAH  ┆ 761       ┆ 25.31716   ┆ 0.0    │
│ 2013 ┆ 4     ┆ 10  ┆ 19   ┆ … ┆ MCO  ┆ 639       ┆ 33.37262   ┆ 0.14   │
│ …    ┆ …     ┆ …   ┆ …    ┆ … ┆ …    ┆ …         ┆ …          ┆ …      │
│ 2013 ┆ 9     ┆ 12  ┆ 19   ┆ … ┆ MIA  ┆ 285       ┆ 18.41248   ┆ 0.23   │
│ 2013 ┆ 2     ┆ 17  ┆ 9    ┆ … ┆ BOS  ┆ 284       ┆ 26.46794   ┆ 0.0    │
│ 2013 ┆ 

In [28]:
result = ctx.execute("""
SELECT
  COALESCE(p.manufacturer, 'Unknown') AS manufacturer,
  COALESCE(p.model, 'Unknown') AS model,
  COUNT(*) AS num_flights
FROM flights AS f
LEFT JOIN planes AS p
  ON p.tailnum = f.tailnum
GROUP BY manufacturer, model
ORDER BY num_flights DESC
LIMIT 10
""")
print(result)

shape: (10, 3)
┌──────────────────┬─────────────────┬─────────────┐
│ manufacturer     ┆ model           ┆ num_flights │
│ ---              ┆ ---             ┆ ---         │
│ str              ┆ str             ┆ u32         │
╞══════════════════╪═════════════════╪═════════════╡
│ Unknown          ┆ Unknown         ┆ 52606       │
│ AIRBUS           ┆ A320-232        ┆ 31278       │
│ EMBRAER          ┆ EMB-145LR       ┆ 28027       │
│ EMBRAER          ┆ ERJ 190-100 IGW ┆ 23716       │
│ AIRBUS INDUSTRIE ┆ A320-232        ┆ 14553       │
│ EMBRAER          ┆ EMB-145XR       ┆ 14051       │
│ BOEING           ┆ 737-824         ┆ 13809       │
│ BOMBARDIER INC   ┆ CL-600-2D24     ┆ 11807       │
│ BOEING           ┆ 737-7H4         ┆ 10389       │
│ BOEING           ┆ 757-222         ┆ 9150        │
└──────────────────┴─────────────────┴─────────────┘


In [29]:
result = ctx.execute("""
WITH route_stats AS (
  SELECT
    f.origin,
    f.dest,
    COUNT(*) AS total_flights,
    AVG(f.dep_delay) AS avg_dep_delay,
    100.0 * SUM(CASE WHEN f.dep_delay > 30 THEN 1 ELSE 0 END) / COUNT(*) AS pct_delayed_gt_30
  FROM flights AS f
  WHERE f.dep_delay IS NOT NULL
  GROUP BY f.origin, f.dest
)
SELECT
  rs.origin,
  ao.name AS origin_name,
  rs.dest,
  ad.name AS dest_name,
  rs.total_flights,
  rs.avg_dep_delay,
  rs.pct_delayed_gt_30
FROM route_stats AS rs
LEFT JOIN airports AS ao
  ON ao.faa = rs.origin
LEFT JOIN airports AS ad
  ON ad.faa = rs.dest
ORDER BY rs.total_flights DESC
LIMIT 10
""")
print(result)

shape: (10, 7)
┌────────┬────────────────┬──────┬────────────────┬───────────────┬───────────────┬────────────────┐
│ origin ┆ origin_name    ┆ dest ┆ dest_name      ┆ total_flights ┆ avg_dep_delay ┆ pct_delayed_gt │
│ ---    ┆ ---            ┆ ---  ┆ ---            ┆ ---           ┆ ---           ┆ _30            │
│ str    ┆ str            ┆ str  ┆ str            ┆ u32           ┆ f64           ┆ ---            │
│        ┆                ┆      ┆                ┆               ┆               ┆ f64            │
╞════════╪════════════════╪══════╪════════════════╪═══════════════╪═══════════════╪════════════════╡
│ JFK    ┆ John F Kennedy ┆ LAX  ┆ Los Angeles    ┆ 11196         ┆ 8.522508      ┆ 9.88746        │
│        ┆ Intl           ┆      ┆ Intl           ┆               ┆               ┆                │
│ LGA    ┆ La Guardia     ┆ ATL  ┆ Hartsfield     ┆ 10082         ┆ 11.448621     ┆ 12.467764      │
│        ┆                ┆      ┆ Jackson        ┆               ┆         

In [30]:
polars_result = (
    flights
    .filter(pl.col("dep_delay").is_not_null())
    .group_by("origin")
    .agg(pl.col("dep_delay").mean().alias("avg_dep_delay"))
    .sort("avg_dep_delay", descending=True)
)
print(polars_result)

shape: (3, 2)
┌────────┬───────────────┐
│ origin ┆ avg_dep_delay │
│ ---    ┆ ---           │
│ str    ┆ f64           │
╞════════╪═══════════════╡
│ EWR    ┆ 15.107954     │
│ JFK    ┆ 12.112159     │
│ LGA    ┆ 10.346876     │
└────────┴───────────────┘
